# Variables

In [ ]:
# Service Principle Variables
appid = "App ID"
secretid = "Secret"
tenant = "Tenant ID"

# Azure Service Variables
subscription_id="Subscription ID"
resource_group_name = "Resoure Group Name"

# Azure ML Variables
ml_workspace = "Workspace Name"

# Azure IoT Variables
iot_hub_name="IoT Hub Name"
iot_device_id="IoT Device ID"
module_name = "Module Name"

# Azure Key Vault Variables
keyVaultName = "Key Vault Name"
KVUri = "Key Vault URI"
Secret_Name = "Secret Name"

_**(Option) Use Key Valut**_

In [ ]:
# !pip install azure-identity
# !pip install azure-keyvault-secrets

# from azure.identity import ClientSecretCredential
# from azure.mgmt.compute import ComputeManagementClient
# from azure.keyvault.secrets import SecretClient
# from azure.identity import DefaultAzureCredential

# credential = ClientSecretCredential(
#     tenant_id=tenant,
#     client_id=appid,
#     client_secret=secretid
# )

# client = SecretClient(vault_url=KVUri, credential=credential)
# secretName = Secret_Name
# retrieved_secret = client.get_secret(secretName)

# print(f"Your secret is '{retrieved_secret.value}'.")

# Service Principal Authentication

In [ ]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(tenant_id=tenant,
                                    service_principal_id=appid, 
                                    service_principal_password=secretid) 
                                    

# Get Azure ML Workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name=ml_workspace,
                   auth=sp,
                   subscription_id=subscription_id,
                   resource_group=resource_group_name)

# Inferencing Image Configuration

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

execution_file="iot_score.py"
tags = {'area': "iot", 'type': "classification"}
description = "IOT Edge anomaly detection demo"

myenv = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script=execution_file,
                                 conda_file="myenv.yml",
                                 tags = tags,
                                 description = description)

# Create Image on Container Registry

In [ ]:
from azureml.core.model import Model

model_name = 'model.pkl'
image_name = "tempanomalydetection"

model = Model(ws, model_name)
image = Image.create(name = image_name,
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = False)

# Get Image Information from Container Registry

In [ ]:
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(sp,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name)
username = result.username
password = result.passwords[0].value

# Make .json with Deployment Template

In [ ]:
file = open('iot-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

# Register Image to the IoT Hub

In [ ]:
!pip install azure-cli
!pip install azure-iot-hub
!az extension add --name azure-iot

!az login --service-principal --username $appid --password $secretid --tenant $tenant
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json